<h1>Objetivo</h1>
<h5><b>Avaliação:</b></h5><p>Para avaliar a performance da previsão do timestamp datahora, será usada a métrica RMSE. Para avaliar a performance da previsão de latitude e longitude, será usada a métrica Mean Haversine Distance</p>

In [1]:
#Manipulação de arquivos
import os
import zipfile
from glob import glob
import json
import gc
import warnings

#Manipulação e processamento de dados
import pandas as pd
import numpy as np
import cudf
import cupy as cp

#Machine Learning
from cuml.cluster import DBSCAN as cuDBSCAN
from sklearn.cluster import DBSCAN as SklearnDBSCAN
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import torch
import joblib
import optuna
import xgboost as xgb
from cuml.ensemble import RandomForestRegressor as cuRF
from catboost import CatBoostRegressor

#Barra de progresso
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook

#Visualização
import folium
from folium.plugins import HeatMap
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.dates as mdates

#Perfil de memória
from memory_profiler import memory_usage

#Configurações de de visualização no Pandas
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

#Suprimir mensagens de warning
warnings.filterwarnings("ignore", category=FutureWarning, message=".*fillna with 'method' is deprecated.*")
warnings.filterwarnings("ignore", category=UserWarning, message=".*Could not infer format, so each element will be parsed individually.*")
warnings.filterwarnings("ignore", category=FutureWarning, message="'squared' is deprecated")
warnings.filterwarnings("ignore", category=UserWarning, message=".*The tree method `gpu_hist` is deprecated since 2.0.0.*")
warnings.filterwarnings("ignore", category=UserWarning, message=".*Falling back to prediction using DMatrix due to mismatched devices.*")


In [ ]:
#Caminhos das pastas
base_dir = os.getcwd()
dados_dir = os.path.join(base_dir, 'dados')
intermediarios_dir = os.path.join(dados_dir, 'intermediarios')
processados_dir = os.path.join(dados_dir, 'processados')

#Criar diretórios se não existirem
os.makedirs(intermediarios_dir, exist_ok=True)
os.makedirs(processados_dir, exist_ok=True)

#Lista global para armazenar os DataFrames processados
final_df_list = []

#Dicionário para armazenar o sentido atual de cada ônibus
sentidos_atual = {}

<h1>EDA</h1>
<p>Fazendo algumas análises estatísticas e plotagens de dados</p>

In [ ]:
class EDA:
    def __init__(self, file_path):
        #Inicializa a classe com o caminho do arquivo de dados processados.
        self.file_path = file_path  

    #Carrega os dados do arquivo Parquet usando pandas ou cudf.
    def load_data(self, use_cudf=False):
        try:
            if use_cudf:
                return cudf.read_parquet(self.file_path) 
            else:
                return pd.read_parquet(self.file_path) 
        except Exception as e:
            print(f"Erro ao carregar os dados: {e}")
            return None

    #Realiza uma inspeção inicial dos dados, visualizando as primeiras linhas e um resumo estatístico.
    def initial_inspection(self):
        df = self.load_data(use_cudf=False)
        if df is not None:
            print(df.describe()) 
            del df  
            gc.collect() 

    #Analisa as estatísticas dos dados, incluindo a distribuição das velocidades e a contagem de registros por linha e sentido.
    def analyze_statistics(self):
        df = self.load_data(use_cudf=False)
        if df is not None:
            #Plota a distribuição das velocidades
            df['velocidade'].hist(bins=50)  # Plota um histograma
            plt.title('Distribuição de Velocidades')
            plt.xlabel('Velocidade')
            plt.ylabel('Frequência')
            plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))  # Formata o eixo y
            plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))  # Formata o eixo x
            plt.show()

            #Conta registros por linha e sentido e exibe os resultados
            #print(df.groupby(['linha', 'sentido']).size().to_frame('count').reset_index())
            del df 
            gc.collect()

    #Analisa padrões temporais dos dados, incluindo o volume de dados por hora e padrões de movimentação ao longo do tempo.
    def analyze_temporal_patterns(self):
        df = self.load_data(use_cudf=False)
        if df is not None:
            #Extrai a hora do timestamp
            #df['datahora_converted'] = pd.to_datetime(df['datahora'])
            df['hora'] = df['datahora_converted'].dt.hour

            # Filtra as horas entre 8 e 22
            df = df[(df['hora'] >= 8) & (df['hora'] < 22)]

            #Conta registros por hora e plota o volume de dados por hora
            hourly_counts = df.groupby('hora').size().to_frame('count').reset_index()
            hourly_counts.plot(x='hora', y='count', kind='bar')
            plt.title('Volume de Dados por Hora')
            plt.xlabel('Hora do Dia')
            plt.ylabel('Contagem')
            plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))  # Formata o eixo y
            plt.show()

            #Conta registros por dia ao longo do tempo e plota padrões de movimentação
            df['data'] = df['datahora_converted'].dt.date  # Extrai apenas a data
            daily_counts = df.groupby('data').size().to_frame('count').reset_index()
            daily_counts.plot(x='data', y='count', kind='line')
            plt.title('Padrões de Movimentação ao Longo do Tempo')
            plt.xlabel('Data')
            plt.ylabel('Contagem')
            plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))  # Formata o eixo y
            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))  # Formata o eixo x para datas
            plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
            plt.xticks(rotation=45)
            plt.show()

            del df  
            gc.collect()  

    #Analisa padrões geográficos dos dados, plotando um mapa de calor dos trajetos dos ônibus.
    def analyze_geographical_patterns(self, linhas=None):
        try:
            df_cudf = self.load_data(use_cudf=True)
            if df_cudf is not None:
                df_cudf['linha'] = df_cudf['linha'].astype(str)  # Garante que a coluna 'linha' seja string

                if linhas:
                    linhas = list(map(str, linhas))
                    for linha in linhas:
                        if linha not in df_cudf['linha'].to_pandas().tolist():
                            raise ValueError(f"Linha {linha} não encontrada nos dados.")

                    df_cudf = df_cudf[df_cudf['linha'].isin(linhas)]

                num_linhas = len(linhas) if linhas else len(df_cudf['linha'].unique())
                sample_fraction = min(1.0 / num_linhas, 0.01)  # Ajusta a fração de amostra de acordo com o número de linhas
                max_points = min(5000 * num_linhas, 50000)  # Ajusta o número máximo de pontos de acordo com o número de linhas

                #Define o centro do mapa baseado na média de latitude e longitude usando cudf
                map_center = [float(df_cudf['latitude'].mean()), float(df_cudf['longitude'].mean())]
                map_osm = folium.Map(location=map_center, zoom_start=12)  # Cria um mapa centrado na localização média

                #Mostra os dados antes de converter para pandas para evitar problemas de memória
                if len(df_cudf) > max_points:
                    df_sample = df_cudf.sample(frac=sample_fraction)
                    if len(df_sample) > max_points:
                        df_sample = df_sample.sample(n=max_points)
                else:
                    df_sample = df_cudf

                df_pandas = df_sample.to_pandas()

                #Dados para o mapa de calor
                heat_data = [[row['latitude'], row['longitude']] for index, row in df_pandas[['latitude', 'longitude']].iterrows()]
                HeatMap(heat_data).add_to(map_osm)  # Adiciona o mapa de calor ao mapa

                title = 'Mapa de Calor de Todas as Linhas'
                if linhas:
                    title = f'Mapa de Calor das Linhas: {", ".join(map(str, linhas))}'
                map_osm.get_root().html.add_child(folium.Element(f"<h1>{title}</h1>"))

                map_osm.save('bus_routes.html')  # Salva o mapa como um arquivo HTML
                print(f"Mapa de calor salvo como 'bus_routes.html' com {len(df_pandas)} pontos de amostra")
                
                del df_cudf  
                del df_pandas  
                gc.collect()
            else:
                print("Erro ao carregar os dados.")
        except ValueError as ve:
            print(ve)
        except Exception as e:
            print(f"Erro ao gerar o mapa de calor: {e}")

    #Plota os trajetos dos ônibus, locais de garagem e pontos finais com diferentes cores.
    def plot_routes_and_endpoints(self, linhas=None):
        try:
            df_cudf = self.load_data(use_cudf=True)
            if df_cudf is not None:
                df_cudf['linha'] = df_cudf['linha'].astype(str)  # Garante que a coluna 'linha' seja string

                if linhas:
                    linhas = list(map(str, linhas))
                    for linha in linhas:
                        if linha not in df_cudf['linha'].to_pandas().tolist():
                            raise ValueError(f"Linha {linha} não encontrada nos dados.")

                    df_cudf = df_cudf[df_cudf['linha'].isin(linhas)]

                num_linhas = len(linhas) if linhas else len(df_cudf['linha'].unique())
                sample_fraction = min(1.0 / num_linhas, 0.01)  # Ajusta a fração de amostra de acordo com o número de linhas
                max_points = min(5000 * num_linhas, 50000)  # Ajusta o número máximo de pontos de acordo com o número de linhas

                #Define o centro do mapa baseado na média de latitude e longitude usando cudf
                map_center = [float(df_cudf['latitude'].mean()), float(df_cudf['longitude'].mean())]
                map_osm = folium.Map(location=map_center, zoom_start=12)  # Cria um mapa centrado na localização média

                #Mostra os dados antes de converter para pandas para evitar problemas de memória
                if len(df_cudf) > max_points:
                    df_sample = df_cudf.sample(frac=sample_fraction)
                    if len(df_sample) > max_points:
                        df_sample = df_sample.sample(n=max_points)
                else:
                    df_sample = df_cudf

                df_pandas = df_sample.to_pandas()

                #Plota os trajetos normais
                normal_routes = df_pandas[~df_pandas['garagem'] & ~df_pandas['ponto_final']]
                HeatMap([[row['latitude'], row['longitude']] for index, row in normal_routes[['latitude', 'longitude']].iterrows()],
                        name='Trajetos Normais', gradient={0.4: 'blue', 1: 'lightblue'}).add_to(map_osm)

                #Plota os locais de garagem
                garage_routes = df_pandas[df_pandas['garagem']]
                HeatMap([[row['latitude'], row['longitude']] for index, row in garage_routes[['latitude', 'longitude']].iterrows()],
                        name='Locais de Garagem', gradient={0.4: 'red', 1: 'darkred'}).add_to(map_osm)

                #Plota os pontos finais
                end_points = df_pandas[df_pandas['ponto_final']]
                HeatMap([[row['latitude'], row['longitude']] for index, row in end_points[['latitude', 'longitude']].iterrows()],
                        name='Pontos Finais', gradient={0.4: 'yellow', 1: 'gold'}).add_to(map_osm)

                title = 'Mapa de Trajetos, Locais de Garagem e Pontos Finais de Todas as Linhas'
                if linhas:
                    title = f'Mapa de Trajetos, Locais de Garagem e Pontos Finais das Linhas: {", ".join(map(str, linhas))}'
                map_osm.get_root().html.add_child(folium.Element(f"<h1>{title}</h1>"))

                map_osm.save('bus_routes_endpoints.html')  # Salva o mapa como um arquivo HTML
                print(f"Mapa de trajetos e pontos salvos como 'bus_routes_endpoints.html' com {len(df_pandas)} pontos de amostra")

                del df_cudf  #Libera a memória
                del df_pandas  #Libera a memória
                gc.collect()  #Coleta o lixo
            else:
                print("Erro ao carregar os dados.")
        except ValueError as ve:
            print(ve)
        except Exception as e:
            print(f"Erro ao gerar o mapa de trajetos e pontos: {e}")

    #Analisa a distribuição dos pontos finais e pontos de garagem.
    def analyze_endpoints_garage(self):
        df = self.load_data(use_cudf=False)
        if df is not None:
            #Plota pontos finais
            df[df['ponto_final'] == True][['latitude', 'longitude']].plot(kind='scatter', x='longitude', y='latitude')
            plt.title('Distribuição de Pontos Finais')
            plt.show()

            #Plota pontos de garagem
            df[df['garagem'] == True][['latitude', 'longitude']].plot(kind='scatter', x='longitude', y='latitude')
            plt.title('Distribuição de Pontos de Garagem')
            plt.show()

            del df  #Libera a memória
            gc.collect()  #Coleta o lixo

    #Analisa os trajetos dos ônibus para cada linha.
    def analyze_routes(self):
        df = self.load_data(use_cudf=False)
        if df is not None:
            linhas = df['linha'].unique()  # Obtém todas as linhas únicas
            num_linhas = len(linhas)
            num_cols = 3
            num_rows = (num_linhas + num_cols - 1) // num_cols  # Calcula o número de linhas para os subplots
            fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, num_rows * 5))
            axes = axes.flatten()

            for idx, linha in enumerate(linhas):
                trajeto = df[df['linha'] == linha]  # Filtra os dados pela linha
                ax = axes[idx]
                ax.plot(trajeto['longitude'], trajeto['latitude'], label=f'Linha {linha}')  # Plota o trajeto
                ax.set_title(f'Trajeto da Linha {linha}')
                ax.set_xlabel('Longitude')
                ax.set_ylabel('Latitude')
                ax.legend()
                ax.yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))  # Formata o eixo y
                ax.xaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))  # Formata o eixo x

            # Remove subplots vazios
            for idx in range(len(linhas), len(axes)):
                fig.delaxes(axes[idx])

            plt.tight_layout()
            plt.show()

            del df  # Libera a memória
            gc.collect()  # Coleta o lixo

    def check_data_quality(self):
        """
        Verifica a qualidade dos dados, incluindo valores nulos, dados inconsistentes e outliers.
        """
        df = self.load_data(use_cudf=False)
        if df is not None:
            # Verifica valores nulos em cada coluna
            print("Verificando valores nulos...")
            print(df.isnull().sum())

            # Verifica valores não numéricos na coluna 'velocidade'
            print("Verificando dados inconsistentes...")
            problematic_values = df[~df['velocidade'].astype(str).str.isnumeric()]
            if not problematic_values.empty:
                print(f"Valores problemáticos em 'velocidade':\n{problematic_values[['velocidade', 'velocidade']].head()}")

            # Verifica outliers de velocidade
            print("Verificando outliers de velocidade [Velocidade acima de 150km/h]...")
            outliers = df[(df['velocidade'] < 0) | (df['velocidade'] > 150)]
            print(f"Outliers:\n{outliers}")

            del df  # Libera a memória
            gc.collect()  # Coleta o lixo

    def feature_engineering(self, df):
        """
        Cria novas features a partir das informações temporais.
        """
        print("Criando novas features...")
        #df['datahora_converted'] = pd.to_datetime(df['datahora'])
        df['dia_da_semana'] = df['datahora_converted'].dt.dayofweek  # Adiciona a feature 'dia_da_semana'
        df['hora'] = df['datahora_converted'].dt.hour  # Adiciona a feature 'hora'
        print("Novas features criadas: 'dia_da_semana', 'hora'")
        return df


    def run_all(self):
        """
        Executa todos os métodos de análise exploratória de dados em sequência.
        """
        self.initial_inspection()
        self.analyze_statistics()
        self.analyze_temporal_patterns()
        self.analyze_geographical_patterns()
        self.plot_routes_and_endpoints()
        self.analyze_endpoints_garage()
        self.analyze_routes()
        self.check_data_quality()


In [ ]:
eda = EDA(file_path=os.path.join(processados_dir, 'processed_data.parquet'))

In [ ]:
#Realizar inspeção inicial
eda.initial_inspection()

In [ ]:
#Analisar estatísticas
eda.analyze_statistics()

In [ ]:
#Analisar padrões temporais
eda.analyze_temporal_patterns()

In [ ]:
#Analisar padrões geográficos
eda.analyze_geographical_patterns(linhas=["324", "3"])

In [ ]:
#Plotar trajetos e pontos
eda.plot_routes_and_endpoints(linhas=["324", "3"])

In [ ]:
#Analisar pontos finais e pontos de garagem
eda.analyze_endpoints_garage()

In [ ]:
#Analisar trajetos dos ônibus
eda.analyze_routes()


In [ ]:
#Verificar qualidade dos dados
eda.check_data_quality()

<h1>Treinando 2 horas de dados consecutivas para prever a próxima hora</h1>

<h3>Extraindo os dados dos arquivos .zip e concatenando os arquivos .json:</h3>
<p>Nessa parte do código a ideia é extrair os arquivos .json dos arquivos .zip. Cada arquivo .zip contém 2 tipos de dados: dados históricos e dados de teste. Portanto, ao final, todos os arquivos de dados históricos foram concatenados em 1 arquivo e todos os arquivos de teste foram concatenados em outro arquivo.</p>
<h3>2 tipos de arquivos de teste:</h3>
<p>Existem 2 arquivos de teste: Um para testar previsão de datahora e outro para testar a previsão de latitude e longitude. Portanto, 2 arquivos de teste foram criados</p>
<h3>Filtragem Inicial:</h3>
<p>Para atender as demandas do enunciado do professor Zimbrão, algumas filtragens iniciais foram feitas:
<li>apenas linhas específicas que serão analizadas</li>
<li>considerei apenas os limites de velocidade entre 0 e 250</li>
<li>considerei apenas os limites de latitude e longitude possíveis: entre -90 e 90 e entre -180 e 180</li>
<li>eu adicionei os 2 últimos dígitos de cada arquivo .json em uma nova coluna para ambos os datasets de treino e teste e chamei essa nova coluna de hour_from_file. O objetivo é usar essa coluna como referência na hora de treinar as 2 últimas horas para prever a hora seguinte</li>

</p>

In [ ]:
'''
A classe `FinalDataProcessor` é responsável por processar arquivos de dados finais. Ela define o diretório 
base e cria diretórios para arquivos intermediários e processados, além de definir um tamanho de chunk 
padrão.
'''
class FinalDataProcessor:
    def __init__(self, base_dir):
        self.base_dir = os.path.join(base_dir, 'dados_finais')
        self.intermediarios_teste_dir = os.path.join(self.base_dir, 'intermediarios_teste')
        self.processados_teste_dir = os.path.join(self.base_dir, 'processados_teste')
        self.chunk_size = 100000

        os.makedirs(self.intermediarios_teste_dir, exist_ok=True)
        os.makedirs(self.processados_teste_dir, exist_ok=True)

    '''
    Recebe uma lista de arquivos zip e extrai cada um deles no diretório de 
    arquivos intermediários de teste. Em seguida, processa os arquivos JSON extraídos.
    '''
    def extract_zip_files(self, zip_files):
        '''
        Loop através de cada arquivo zip na lista `zip_files`.
        '''
        for zip_file in tqdm(zip_files, desc="Extracting zip files"):
            with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                zip_ref.extractall(self.intermediarios_teste_dir)
                date_from_file = '-'.join(os.path.basename(zip_file).split('-')[1:4]).split('.')[0]
                print(f"DATE FROM FILE: {date_from_file}")
                self.process_json_files(date_from_file)

    '''
    Chama os métodos para processar e salvar arquivos de teste e concatenar e salvar arquivos de treino, 
    com base na data extraída do nome do arquivo.
    '''
    def process_json_files(self, date_from_file):
        self.process_and_save_test_files(date_from_file)
        self.concatenate_and_save_train_files(date_from_file)

    '''
    Lê um arquivo JSON e o converte em um DataFrame do cuDF. cuDF é usado para acelerar o processo, usando
    a aceleração por GPU
    '''
    def process_json(self, json_path):
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        df = cudf.DataFrame.from_pandas(pd.DataFrame(data))
        return df

    '''
    Faz o pré-processamento do DataFrame de treino, adicionando colunas com informações de data e hora 
    extraídas do nome do arquivo, filtrando linhas válidas e convertendo e interpolando colunas específicas.
    '''
    def preprocess_train_df(self, df, json_path, date_from_file):
        hour_from_file_train = int(json_path.split('_')[-1].split('.')[0])
        df['hour_from_file_train'] = hour_from_file_train
        df['date_hour_from_file_train'] = f"{date_from_file}_{hour_from_file_train}"
        df['date_from_file'] = date_from_file

        linhas_validas = [483, 864, 639, 3, 309, 774, 629, 371, 397, 100, 838, 315, 624, 388, 918, 665, 328, 497, 878, 355, 138, 606, 457, 550, 803, 917, 638, 2336, 399, 298, 867, 553, 565, 422, 756, 186012003, 292, 554, 634, 232, 415, 2803, 324, 852, 557, 759, 343, 779, 905, 108]
        df = df[df['linha'].astype(str).isin(map(str, linhas_validas))]

        if 'datahoraenvio' in df.columns:
            df['datahoraenvio'] = cudf.to_datetime(df['datahoraenvio'].astype('int64'), unit='ms')
        if 'datahora' in df.columns:
            df['datahora_converted'] = cudf.to_datetime(df['datahora'].astype('int64'), unit='ms')

        if 'latitude' in df.columns:
            df['latitude'] = df['latitude'].str.replace(',', '.').astype('float32')
        if 'longitude' in df.columns:
            df['longitude'] = df['longitude'].str.replace(',', '.').astype('float32')

        '''
        Faz a interpolação e preenche os valores nulos.
        '''
        while df['latitude'].isnull().any() or df['longitude'].isnull().any():
            df['latitude'] = df['latitude'].interpolate().ffill().bfill()
            df['longitude'] = df['longitude'].interpolate().ffill().bfill()

        if 'velocidade' in df.columns:
            df['velocidade'] = df['velocidade'].astype('int32')
            df = df[(df['velocidade'] >= 0) & (df['velocidade'] <= 250)]
        if 'latitude' in df.columns and 'longitude' in df.columns:
            df = df[df['latitude'].between(-90, 90) & df['longitude'].between(-180, 180)]

        return df

    '''
    Faz o pré-processamento do DataFrame de teste, adicionando colunas com informações de data e 
    hora extraídas do nome do arquivo e convertendo a coluna `datahora`.
    '''
    def preprocess_test_df(self, df, json_path, date_from_file):
        hour_from_file_test = int(json_path.split('_')[-1].split('.')[0])
        df['hour_from_file_test'] = hour_from_file_test
        df['date_hour_from_file_test'] = f"{date_from_file}_{hour_from_file_test}"
        df['date_from_file'] = date_from_file

        if 'datahora' in df.columns:
            df['datahora_converted'] = cudf.to_datetime(df['datahora'].astype('int64'), unit='ms')

        return df

    '''
    Processa arquivos de teste encontrados no diretório intermediário e salva os DataFrames processados 
    como arquivos CSV no diretório de processados.
    '''
    def process_and_save_test_files(self, date_from_file):
        '''
        Lista de arquivos de teste no diretório intermediário.
        '''
        test_files = []
        for root, _, files in os.walk(self.intermediarios_teste_dir):
            for file in files:
                if file.startswith('teste') and file.endswith('.json'):
                    test_files.append(os.path.join(root, file))

        print(f"Found {len(test_files)} test files to process for date: {date_from_file}")

        '''
        Loop através de cada arquivo de teste encontrado.
        '''
        for test_file in tqdm(test_files, desc=f"Processing test files for {date_from_file}"):
            print(f"Processing file: {test_file}")
            df = self.process_json(test_file)
            df = self.preprocess_test_df(df, test_file, date_from_file)
            hour_from_file_test = int(test_file.split('_')[-1].split('.')[0])
            output_csv = os.path.join(self.processados_teste_dir, f'test_data_{date_from_file}_{hour_from_file_test}.csv')
            print(f"Saving processed data to: {output_csv}")
            df.to_pandas().to_csv(output_csv, index=False)

    '''
    Processa e concatena arquivos de treino encontrados no diretório intermediário, 
    salvando os DataFrames concatenados como arquivos CSV no diretório de processados.
    '''
    def concatenate_and_save_train_files(self, date_from_file):
        '''
        Lista de arquivos de treino no diretório intermediário.
        '''
        train_files = []
        for root, _, files in os.walk(self.intermediarios_teste_dir):
            for file in files:
                if file.startswith('2024') and file.endswith('.json'):
                    train_files.append(os.path.join(root, file))

        train_files = sorted(train_files)
        '''
        Itera sobre os arquivos de treinos de 2 em 2. A razão disso é porque cada arquivo Json representa
        1 hora. Como eu precisarei treinar 2 horas consecutivas, irei criar arquivos csv concatenando
        2 jsons em um único csv. Por exemplo, o arquivo 2024-05-16_06.json e 2024-05-16_07.json, será 
        transformado em um único arquivo train_data_2024-05-16_06_07.csv
        '''
        for i in tqdm(range(0, len(train_files), 2), desc=f"Processing train files for {date_from_file}"):
            if i + 1 < len(train_files):
                df1 = self.process_json(train_files[i])
                df2 = self.process_json(train_files[i + 1])
                df1 = self.preprocess_train_df(df1, train_files[i], date_from_file)
                df2 = self.preprocess_train_df(df2, train_files[i + 1], date_from_file)
                concatenated_df = cudf.concat([df1, df2], ignore_index=True)
                hour_from_file_train1 = int(train_files[i].split('_')[-1].split('.')[0])
                hour_from_file_train2 = int(train_files[i + 1].split('_')[-1].split('.')[0])
                output_csv = os.path.join(self.processados_teste_dir, f'train_data_{date_from_file}_{hour_from_file_train1}_{hour_from_file_train2}.csv')
                print(f"Saving processed data to: {output_csv}")
                concatenated_df.to_pandas().to_csv(output_csv, index=False)

    '''
    Inicia a execução de todas as funções
    '''
    def run(self):
        zip_files = glob(os.path.join(self.base_dir, '*.zip'))
        if not zip_files:
            raise FileNotFoundError("No zip files found in the specified directory.")

        '''
        Loop através de cada arquivo zip encontrado.
        '''
        for zip_file in zip_files:
            with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                zip_ref.extractall(self.intermediarios_teste_dir)
                date_from_file = '-'.join(os.path.basename(zip_file).split('-')[1:4]).split('.')[0]
                print(f"DATE FROM FILE: {date_from_file}")
                self.process_json_files(date_from_file)
                '''
                Limpa o diretório intermediário para evitar misturas de arquivos de diferentes arquivos zip.
                '''
                for root, _, files in os.walk(self.intermediarios_teste_dir):
                    for file in files:
                        os.remove(os.path.join(root, file))

'''
Executa o código
'''
if __name__ == "__main__":
    base_dir = os.getcwd()
    processor = FinalDataProcessor(base_dir)
    processor.run()


<h3>Pre processing the data</h3>
<p>Agora eu estou aplicando alguns pre processamentos, como a filtragem por linhas ( por alguma razão, a filtragem anterior não funcionou), a diminuição do tamanho da palavra do tipo de dados para tornar o arquivo final menor, e a exclusão de outliers. Outliers são todos os pontos de latitude e longitude que fogem da rota usual da linha ao longo do dia.Outliers incluem garagens e pontos finais. </p>

In [ ]:
'''
Lista de linhas válidas de acordo com o enunciado.
'''
linhas_validas = [
    483, 864, 639, 3, 309, 774, 629, 371, 397, 100, 838, 315, 624, 388, 918, 665, 328, 497, 878, 355, 138, 606, 457, 
    550, 803, 917, 638, 2336, 399, 298, 867, 553, 565, 422, 756, 186012003, 292, 554, 634, 232, 415, 2803, 324, 852, 
    557, 759, 343, 779, 905, 108
]

'''
Classe responsável pelo pré-processamento de dados, incluindo carregamento, conversão de tipos, 
filtragem de dados, remoção de outliers e salvamento de arquivos intermediários e finais.
'''
class DataPreprocessing:
    def __init__(self, file_path=None, intermediarios_dir=None, processados_dir=None, output_filename=None, chunk_size=1000000):
        self.file_path = file_path
        self.intermediarios_dir = intermediarios_dir
        self.processados_dir = processados_dir
        self.cleaned_file = os.path.join(intermediarios_dir, f'cleaned_{output_filename}') if intermediarios_dir and output_filename else None
        self.prepared_file = os.path.join(processados_dir, 'treated', output_filename) if processados_dir and output_filename else None
        self.chunk_size = chunk_size

    def load_data(self):
        if self.file_path.endswith('.csv'):
            return cudf.read_csv(self.file_path)
        elif self.file_path.endswith('.parquet'):
            return cudf.read_parquet(self.file_path)
        else:
            raise ValueError("Unsupported file format")

    def save_intermediate_data(self, df, filename):
        df.to_parquet(filename)

    '''
    Conversão de dados necessária.
    '''
    def convert_dtypes(self, df):
        if "latitude" in df.columns and (df["latitude"].dtype == 'object' or df["latitude"].dtype.name == 'category'):
            df["latitude"] = df["latitude"].str.replace(',', '.').astype('float32')
        if "longitude" in df.columns and (df["longitude"].dtype == 'object' or df["longitude"].dtype.name == 'category'):
            df["longitude"] = df["longitude"].str.replace(',', '.').astype('float32')
        if "datahora" in df.columns:
            df['datahora'] = df['datahora'].astype('int64')
        if "linha" in df.columns:
            df['linha'] = df['linha'].astype('int32')
        if "ordem" in df.columns:
            df['ordem'] = df['ordem'].astype('object')
        if "velocidade" in df.columns:
            df['velocidade'] = df['velocidade'].astype('int32')
        if "datahoraservidor" in df.columns:
            df['datahoraservidor'] = df['datahoraservidor'].astype('int64')
        if "dia_da_semana" in df.columns:
            df['dia_da_semana'] = df['dia_da_semana'].astype('int32')
        if "hora" in df.columns:
            df['hora'] = df['hora'].astype('int16')
        if "diff_timestamp" in df.columns:
            df['diff_timestamp'] = df['diff_timestamp'].astype('float64')
        if "latitude_diff" in df.columns:
            df['latitude_diff'] = df['latitude_diff'].astype('float32')
        if "longitude_diff" in df.columns:
            df['longitude_diff'] = df['longitude_diff'].astype('float32')
        if "distancia" in df.columns:
            df['distancia'] = df['distancia'].astype('float32')
        return df

    '''
    Filtra o DataFrame mantendo apenas as linhas válidas de acordo com o enunciado.
    '''
    def filter_valid_linhas(self, df):
        if "linha" in df.columns:
            df = df[df['linha'].astype(str).isin(map(str, linhas_validas))]
        return df

    '''
    Remove outliers dos dados usando o algoritmo DBSCAN. 
    Para identificar outliers, eu verifiquei a rota que uma mesma linha faz ao longo do tempo.
    Ou seja, eu verifiquei sua latitude e longitude ao longo do tempo. Tudo que foge dessa rota,
    eu considerei como outliers. Isso inclui garagens de pontos finais.
    '''
    def remove_outliers(self, df, eps=0.001, min_samples=10):
        if os.path.exists(self.cleaned_file):
            df = cudf.read_parquet(self.cleaned_file)
            if 'datahora' in df.columns:
                return df

        if 'datahora_converted' not in df.columns and 'datahora' in df.columns:
            df['datahora_converted'] = cudf.to_datetime(df['datahora'], unit='ms')

        if 'linha' not in df.columns or 'ordem' not in df.columns:
            print(df.head(2))
            raise KeyError("Necessary columns are not present in the data.")

        if 'latitude' not in df.columns or 'longitude' not in df.columns:
            return df

        '''
        Obtém as linhas únicas e converte para pandas.
        '''
        lines = df['linha'].unique().to_pandas()
        cleaned_dfs = []

        '''
        Loop através de cada linha única.
        '''
        for linha in tqdm(lines, desc="Lines", leave=False, dynamic_ncols=True):
            line_data = df[df['linha'] == linha].copy()
            if len(line_data) < min_samples:
                cleaned_dfs.append(line_data)
                continue

            if 'latitude' in line_data.columns and 'longitude' in line_data.columns:
                other_columns = [col for col in line_data.columns if col not in ['latitude', 'longitude']]
                line_data_pd = line_data[['latitude', 'longitude']].to_pandas()
                db = DBSCAN(eps=eps, min_samples=min_samples)
                labels = db.fit_predict(line_data_pd)

                line_data['labels'] = labels
                non_outliers = line_data[line_data['labels'] != -1].copy()
                non_outliers = non_outliers.drop(columns=['labels'])

                '''
                Mescla os dados não outliers com outras colunas.
                '''
                for col in other_columns:
                    non_outliers[col] = line_data[col].loc[non_outliers.index]

                cleaned_dfs.append(non_outliers)

        '''
        Concatena todos os DataFrames limpos e salva os dados intermediários.
        '''
        if cleaned_dfs:
            cleaned_df = cudf.concat(cleaned_dfs, ignore_index=True)
            self.save_intermediate_data(cleaned_df, self.cleaned_file)
        else:
            raise ValueError("No objects to concatenate")

        return cleaned_df

    '''
    Realiza o pré-processamento completo dos dados, incluindo carregamento, conversão de tipos, 
    filtragem de linhas válidas, remoção de outliers e salvamento dos dados processados.
    '''
    def preprocess(self, remove_outliers=False):
        print(f"Processing file: {self.file_path}")
        df = self.load_data()
        df = self.convert_dtypes(df)
        df = self.filter_valid_linhas(df)
        if remove_outliers and 'latitude' in df.columns and 'longitude' in df.columns:
            df = self.remove_outliers(df)
        os.makedirs(os.path.dirname(self.prepared_file), exist_ok=True)
        df.to_parquet(self.prepared_file)
        df.to_csv(self.prepared_file.replace('.parquet', '.csv'), index=False)
        print(f"Processed and saved file: {self.prepared_file}")
        return df


In [ ]:
if __name__ == "__main__":
    print("Starting script execution...")
    base_dir = os.getcwd()
    print(f"Base directory: {base_dir}")

    #Gera os paths necessários
    base_final_dir = os.path.join(base_dir, 'dados_finais')
    intermediarios_teste_dir = os.path.join(base_final_dir, 'intermediarios_teste')
    processados_teste_dir = os.path.join(base_final_dir, 'processados_teste')
    print(f"Base final directory: {base_final_dir}")
    print(f"Intermediarios teste directory: {intermediarios_teste_dir}")
    print(f"Processados teste directory: {processados_teste_dir}")

    os.makedirs(os.path.join(processados_teste_dir, 'treated'), exist_ok=True)

    #Processa os arquivos de teste e treino separadamente
    for root, _, files in os.walk(processados_teste_dir):
        print(f"Current directory: {root}")
        print(f"Files: {files}")
        for file in tqdm(files, desc="Processing files", leave=False, dynamic_ncols=True):
            print(f"Processing file: {file}")
            if file.startswith('train_data_') and file.endswith('.csv'):
                csv_file_path = os.path.join(root, file)
                parquet_file_path = csv_file_path.replace('.csv', '.parquet')
                print(f"Converting {csv_file_path} to {parquet_file_path}")

                #Converte CSV para Parquet
                df = pd.read_csv(csv_file_path)
                df.to_parquet(parquet_file_path)

                #Pré processamento feito aqui
                treino_preprocessing = DataPreprocessing(
                    file_path=parquet_file_path,
                    intermediarios_dir=intermediarios_teste_dir,
                    processados_dir=processados_teste_dir,
                    output_filename=file.replace('.csv', '.parquet')
                )
                treino_preprocessing.preprocess(remove_outliers=True)

            if file.startswith('test_data_') and file.endswith('.csv'):
                csv_file_path = os.path.join(root, file)
                parquet_file_path = csv_file_path.replace('.csv', '.parquet')
                print(f"Converting {csv_file_path} to {parquet_file_path}")

                #Converte CSV para Parquet
                df = pd.read_csv(csv_file_path)
                df.to_parquet(parquet_file_path)

                #Pré processamento para as features de  latitude and longitude
                if 'latitude' in df.columns and 'longitude' in df.columns:
                    lat_long_preprocessing = DataPreprocessing(
                        file_path=parquet_file_path,
                        intermediarios_dir=intermediarios_teste_dir,
                        processados_dir=processados_teste_dir,
                        output_filename=file.replace('test_data_', 'treated_test_lat_long_').replace('.csv', '.parquet')
                    )
                    lat_long_preprocessing.preprocess(remove_outliers=False)

                #Pré processamento para as features de datahora
                if 'datahora' in df.columns:
                    datahora_preprocessing = DataPreprocessing(
                        file_path=parquet_file_path,
                        intermediarios_dir=intermediarios_teste_dir,
                        processados_dir=processados_teste_dir,
                        output_filename=file.replace('test_data_', 'treated_test_datahora_').replace('.csv', '.parquet')
                    )
                    datahora_preprocessing.preprocess(remove_outliers=False)

    print("Script execution finished.")

<h3>Processing the data and creating the features</h3>
<p>A estratégia abaixo divide os arquivos de treino em df_datahora e df_lat_long. O objetivo é treinar arquivos separados para cada previsão que precisa ser feita. Além do mais, encoder específico para a coluna ordem foi criada, pois eu preciso transformar essa coluna em número mas sem alterar muito os seus valores. Também foram criadas novas features importantes, especialmente a velocidade média tanto para os datasets de treino quanto para os datasets de teste</p>

In [ ]:
#Usar a função convert_dtypes diretamente da classe Preprocessing
def convert_dtypes(df):
    if "latitude" in df.columns and (df["latitude"].dtype == 'object' or df["latitude"].dtype.name == 'category'):
        df["latitude"] = df["latitude"].str.replace(',', '.').astype('float32')
    if "longitude" in df.columns and (df["longitude"].dtype == 'object' or df["longitude"].dtype.name == 'category'):
        df["longitude"] = df["longitude"].str.replace(',', '.').astype('float32')
    if "datahora" in df.columns:
        df['datahora'] = df['datahora'].astype('int64')
    if "linha" in df.columns:
        df['linha'] = df['linha'].astype('int32')
    if "ordem" in df.columns:
        df['ordem'] = df['ordem'].astype('object')
    if "velocidade" in df.columns:
        df['velocidade'] = df['velocidade'].astype('int32')
    if "datahoraservidor" in df.columns:
        df['datahoraservidor'] = df['datahoraservidor'].astype('int64')
    if "day_of_week" in df.columns:
        df['day_of_week'] = df['day_of_week'].astype('int32')
    if "hour" in df.columns:
        df['hour'] = df['hour'].astype('int16')
    if "minute" in df.columns:
        df['minute'] = df['minute'].astype('int16')
    if "diff_timestamp" in df.columns:
        df['diff_timestamp'] = df['diff_timestamp'].astype('float64')
    if "latitude_diff" in df.columns:
        df['latitude_diff'] = df['latitude_diff'].astype('float32')
    if "longitude_diff" in df.columns:
        df['longitude_diff'] = df['longitude_diff'].astype('float32')
    if "distancia" in df.columns:
        df['distancia'] = df['distancia'].astype('float32')
    return df

'''
Remove colunas desnecessárias e divide o DataFrame em dois: um para dados de data e hora, e outro para 
dados de latitude e longitude.
'''
def preprocess_and_split(df):
    columns_to_drop = ['datahoraservidor', 'datahoraenvio', 'date_hour_from_file_test']
    df_datahora = df.drop(columns=columns_to_drop, errors='ignore')
    df_lat_long = df.drop(columns=columns_to_drop, errors='ignore')
    return df_datahora, df_lat_long

'''
Codifica a coluna `ordem`, transformando valores que começam com uma letra em um número correspondente 
e mantendo os demais valores como inteiros.
'''
def custom_encode_ordem(df, column='ordem'):
    def encode_value(val):
        if isinstance(val, str) and val[0].isalpha():
            letter = val[0].upper()
            number = ord(letter) - ord('A') + 1
            encoded_val = str(number) + val[1:]
            return int(encoded_val)
        elif str(val).isdigit():
            return int(val)
        return val

    df[column] = df[column].apply(encode_value).astype('int32')
    return df

'''
Cria novas features no DataFrame com base em colunas existentes, como `datahora`, `latitude` e `longitude`.
'''
def apply_feature_engineering(df, target):
    try:
        print(f"Creating new features for {target}...")
        if 'datahora' in df.columns:
            df['datahora_converted'] = pd.to_datetime(df['datahora'], unit='ms')

        if 'datahora_converted' in df.columns and 'linha' in df.columns and 'ordem' in df.columns:
            df = df.sort_values(by=['linha', 'ordem', 'datahora_converted'])
            df['day_of_week'] = df['datahora_converted'].dt.weekday
            df['hour'] = df['datahora_converted'].dt.hour
            df['minute'] = df['datahora_converted'].dt.minute
            df['diff_timestamp'] = df.groupby(['linha', 'ordem'])['datahora_converted'].diff().astype('int64') / 10**6

        if 'latitude' in df.columns and 'longitude' in df.columns:
            df['latitude_diff'] = df.groupby(['linha', 'ordem'])['latitude'].diff().ffill().bfill()
            df['longitude_diff'] = df.groupby(['linha', 'ordem'])['longitude'].diff().ffill().bfill()

            lat1 = np.radians(df['latitude'])
            lon1 = np.radians(df['longitude'])
            lat2 = np.radians(df['latitude'] + df['latitude_diff'])
            lon2 = np.radians(df['longitude'] + df['longitude_diff'])

            dlat = lat2 - lat1
            dlon = lon2 - lon1

            a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
            c = 2 * np.arcsin(np.sqrt(a))
            r = 6371
            df['distancia'] = c * r * 1000

        print(f"New features created for {target}: 'day_of_week', 'hour', 'minute', 'diff_timestamp', 'latitude_diff', 'longitude_diff', 'distancia'")
    except Exception as e:
        print(f"Error in creating new features: {e}")
        raise

    try:
        print(f"Encoding categorical features for {target}...")
        df = custom_encode_ordem(df)
    except Exception as e:
        print(f"Error in encoding categorical features: {e}")
        raise

    return df

'''
Adiciona a velocidade média ao DataFrame de treino e teste, com base em colunas como `datahora`, 
`latitude` e `longitude`.
'''
def add_avg_velocity(train_df, test_df, target):
    train_df = convert_dtypes(train_df.copy())
    test_df = convert_dtypes(test_df.copy())

    train_df = custom_encode_ordem(train_df)
    test_df = custom_encode_ordem(test_df)

    '''
    Como os datasets de teste não possuem a columa velocidade, precisei criar alguns mecanismos para 
    que eu pudesse adicionar a coluna de velocidade média nos datasets de teste. Adicionei a coluna
    velocidade média no dataset de treino agroupando por linha e ordem. Tentei agrupar por algumas 
    features de datahora também, mas estava dando erro. Eu extraio esses valores agrupados para uma 
    variável chamada avg_speed e então eu faço merge nos datasets de teste. 
    '''
    if target == 'datahora':
        train_df['datahora_converted'] = pd.to_datetime(train_df['datahora'], unit='ms')
        test_df['datahora_converted'] = pd.to_datetime(test_df['datahora'], unit='ms')
        if 'datahora_converted' in train_df.columns:
            avg_speed = train_df.groupby(['linha', 'ordem']).agg({'velocidade': 'mean'}).reset_index()
            avg_speed.columns = ['linha', 'ordem', 'velocidade_for_datahora']

            train_df = train_df.merge(avg_speed, on=['linha', 'ordem'], how='left')
            train_df['velocidade_for_datahora'] = train_df['velocidade_for_datahora'].ffill().bfill()

        if 'datahora_converted' in test_df.columns:
            test_df = test_df.merge(avg_speed, on=['linha', 'ordem'], how='left')
            print("Test DataFrame after merge:")
            print(test_df.head())
            test_df['velocidade_for_datahora'] = test_df['velocidade_for_datahora'].ffill().bfill()
            test_df = test_df.rename(columns={'velocidade_for_datahora': 'avg_speed'})
            train_df = train_df.rename(columns={'velocidade_for_datahora': 'avg_speed'})
            
    elif target == 'lat_long':
        if 'latitude' in train_df.columns and 'longitude' in train_df.columns:
            train_df['lat_temp'] = (train_df['latitude'] * 100).astype(int)
            train_df['lon_temp'] = (train_df['longitude'] * 100).astype(int)

            avg_speed = train_df.groupby(['linha', 'ordem', 'lat_temp', 'lon_temp']).agg({'velocidade': 'mean'}).reset_index()
            avg_speed.columns = ['linha', 'ordem', 'lat_temp', 'lon_temp', 'velocidade_for_lat_long']

            train_df = train_df.merge(avg_speed, on=['linha', 'ordem', 'lat_temp', 'lon_temp'], how='left')
            train_df['velocidade_for_lat_long'] = train_df['velocidade_for_lat_long'].ffill().bfill()

        if 'latitude' in test_df.columns and 'longitude' in test_df.columns:
            test_df['lat_temp'] = (test_df['latitude'] * 100).astype(int)
            test_df['lon_temp'] = (test_df['longitude'] * 100).astype(int)

            test_df = test_df.merge(avg_speed, on=['linha', 'ordem', 'lat_temp', 'lon_temp'], how='left')
            test_df['velocidade_for_lat_long'] = test_df['velocidade_for_lat_long'].ffill().bfill()

            test_df = test_df.drop(columns=['lat_temp', 'lon_temp'])
            train_df = train_df.drop(columns=['lat_temp', 'lon_temp'])

            test_df = test_df.rename(columns={'velocidade_for_lat_long': 'avg_speed'})
            train_df = train_df.rename(columns={'velocidade_for_lat_long': 'avg_speed'})

    return train_df, test_df

'''
Ordena as colunas do DataFrame, mantendo colunas iniciais e finais especificadas em suas respectivas 
posições.
'''
def sort_columns(df, initial_columns=None, final_columns=None):
    if initial_columns is None:
        initial_columns = []
    if final_columns is None:
        final_columns = []

    all_columns = list(df.columns)
    columns_to_sort = [col for col in all_columns if col not in initial_columns + final_columns]
    sorted_columns = sorted(columns_to_sort)
    new_order = initial_columns + sorted_columns + final_columns

    df = df[new_order]

    return df

if __name__ == "__main__":
    base_dir = os.getcwd()
    base_final_dir = os.path.join(base_dir, 'dados_finais')
    processados_teste_dir = os.path.join(base_final_dir, 'processados_teste', 'treated')
    feature_modeling_dir = os.path.join(base_final_dir, 'feature_modeling')

    os.makedirs(feature_modeling_dir, exist_ok=True)

    try:
        print("Loading datasets...")
        train_files = [f for f in os.listdir(processados_teste_dir) if f.startswith('train_data_') and f.endswith('.csv')]
        test_datahora_files = [f for f in os.listdir(processados_teste_dir) if f.startswith('treated_test_datahora_') and f.endswith('.csv')]
        test_lat_long_files = [f for f in os.listdir(processados_teste_dir) if f.startswith('treated_test_lat_long_') and f.endswith('.csv')]

        '''
        Processa cada arquivo de treino, aplicando pré-processamento e engenharia de features.
        '''
        for train_file in tqdm(train_files, desc="Processing training files"):
            try:
                print(f"Processing file: {train_file}")
                train_df = pd.read_csv(os.path.join(processados_teste_dir, train_file))

                train_datahora_df, train_lat_long_df = preprocess_and_split(train_df)

                print("Adding average velocity to training data...")
                train_datahora_df, _ = add_avg_velocity(train_datahora_df, train_datahora_df.copy(), 'datahora')
                train_lat_long_df, _ = add_avg_velocity(train_lat_long_df, train_lat_long_df.copy(), 'lat_long')

                train_datahora_df = apply_feature_engineering(train_datahora_df, 'datahora')
                train_lat_long_df = apply_feature_engineering(train_lat_long_df, 'lat_long')

                print("Dropping rows with null values in train_datahora dataframe...")
                print(f"Shape before dropna: {train_datahora_df.shape}")
                train_datahora_df = train_datahora_df.dropna(subset=['diff_timestamp', 'avg_speed'])
                print(f"Shape after dropna: {train_datahora_df.shape}")
                if train_datahora_df.empty:
                    raise ValueError(f"DataFrame train_datahora for file {train_file} is empty after dropping null values.")

                print("Dropping rows with null values in train_lat_long dataframe...")
                print(f"Shape before dropna: {train_lat_long_df.shape}")
                train_lat_long_df = train_lat_long_df.dropna(subset=['latitude_diff', 'longitude_diff'])
                print(f"Shape after dropna: {train_lat_long_df.shape}")
                if train_lat_long_df.empty:
                    raise ValueError(f"DataFrame train_lat_long for file {train_file} is empty after dropping null values.")

                train_datahora_df.to_csv(os.path.join(feature_modeling_dir, f'preprocessed_{train_file.replace(".csv", "_datahora.csv")}'), index=False)
                train_lat_long_df.to_csv(os.path.join(feature_modeling_dir, f'preprocessed_{train_file.replace(".csv", "_lat_long.csv")}'), index=False)

            except Exception as e:
                print(f"Error processing file {train_file}: {e}")

        '''
        Processa cada arquivo de teste `datahora`, aplicando pré-processamento e engenharia de features.
        '''
        for test_file in tqdm(test_datahora_files, desc="Processing test datahora files"):
            try:
                print(f"Processing test datahora file: {test_file}")
                test_datahora_df = pd.read_csv(os.path.join(processados_teste_dir, test_file))

                train_datahora_df = pd.read_csv(os.path.join(processados_teste_dir, train_files[0]))  # Read the first train file for adding avg_velocity

                _, test_datahora_df = add_avg_velocity(train_datahora_df, test_datahora_df, 'datahora')
                test_datahora_df = apply_feature_engineering(test_datahora_df, 'datahora')

                print("Dropping rows with null values in test_datahora dataframe...")
                print(f"Shape before dropna: {test_datahora_df.shape}")
                test_datahora_df = test_datahora_df.dropna(subset=['diff_timestamp', 'avg_speed'])
                print(f"Shape after dropna: {test_datahora_df.shape}")
                if test_datahora_df.empty:
                    raise ValueError(f"DataFrame test_datahora for file {test_file} is empty after dropping null values.")

                test_datahora_df.to_csv(os.path.join(feature_modeling_dir, f'preprocessed_{test_file}'), index=False)

            except Exception as e:
                print(f"Error processing file {test_file}: {e}")

        '''
        Processa cada arquivo de teste `lat_long`, aplicando pré-processamento e engenharia de features.
        '''
        for test_file in tqdm(test_lat_long_files, desc="Processing test lat_long files"):
            try:
                print(f"Processing test lat_long file: {test_file}")
                test_lat_long_df = pd.read_csv(os.path.join(processados_teste_dir, test_file))

                train_lat_long_df = pd.read_csv(os.path.join(processados_teste_dir, train_files[0]))  # Read the first train file for adding avg_velocity

                _, test_lat_long_df = add_avg_velocity(train_lat_long_df, test_lat_long_df, 'lat_long')
                test_lat_long_df = apply_feature_engineering(test_lat_long_df, 'lat_long')

                print("Dropping rows with null values in test_lat_long dataframe...")
                print(f"Shape before dropna: {test_lat_long_df.shape}")
                test_lat_long_df = test_lat_long_df.dropna(subset=['latitude_diff', 'longitude_diff'])
                print(f"Shape after dropna: {test_lat_long_df.shape}")
                if test_lat_long_df.empty:
                    raise ValueError(f"DataFrame test_lat_long for file {test_file} is empty after dropping null values.")

                test_lat_long_df.to_csv(os.path.join(feature_modeling_dir, f'preprocessed_{test_file}'), index=False)

            except Exception as e:
                print(f"Error processing file {test_file}: {e}")

        print("Feature engineering and preprocessing complete.")
    except Exception as e:
        print(f"Error in the main processing block: {e}")


<h3>Fazendo Cross Validation</h3>
<p>Aqui irei fazer a validação cruzada. Basicamente, teremos duas previsões a serem feitas de maneira separada. Para isso, irei fazer duas validações cruzadas, testando Random Forest,XGBoost e CatBoost com vários parâmetros. Para cada validação cruzada, o objetivo é encontrar o melhor modelo, com os melhores parâmetros. </p>
<p>No caso da previsão de datahora, a métrica utilizada para a avaliação do melhor modelo é o RMSE. No caso da previsão de latitude, a métrica utilizada para a avaliação do melhor modelo é a média da distância Harversine</p>
<p>Estou usando o Optuna para a validação cruzada e estou usando a aceleração por CUDA. </p>
<p>Após isso, o melhor modelo final é escolhido para cada um das previsões feitas</p>

In [ ]:
'''
Define os caminhos para os diretórios onde serão armazenados os modelos intermediários e finais.
'''
base_path = 'dados_finais/feature_modeling'
intermediate_model_path = 'dados_finais/cross_validation/intermediate_model'
final_model_path = 'dados_finais/cross_validation/final_model'

'''
Garante que os diretórios existem, criando-os se necessário.
'''
os.makedirs(intermediate_model_path, exist_ok=True)
os.makedirs(final_model_path, exist_ok=True)

'''
Função `train_and_evaluate` treina e avalia um modelo com os parâmetros especificados no `trial`. Retorna o modelo treinado e o RMSE.
'''
def train_and_evaluate(X_train, X_test, y_train, y_test, model_name, trial, target_dimension):
    try:
        print(f"Training model: {model_name} with trial parameters: {trial.params}")
        '''
        Verifica qual modelo será treinado (CatBoost, XGBoost ou cuml_rf) e define os parâmetros específicos para cada um.
        '''
        if model_name == "CatBoost":
            model = MultiOutputRegressor(
                CatBoostRegressor(
                    iterations=trial.suggest_int("iterations", 100, 1000),
                    learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
                    depth=trial.suggest_int("depth", 4, 10),
                    silent=True,
                    task_type="GPU" 
                )
            )
        elif model_name == "XGBoost":
            model = MultiOutputRegressor(
                XGBRegressor(
                    n_estimators=trial.suggest_int("n_estimators", 100, 1000),
                    learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
                    max_depth=trial.suggest_int("max_depth", 4, 10),
                    tree_method='hist',
                    device='cuda'
                )
            )
        elif model_name == "cuml_rf":
            if target_dimension == 1:
                model = cuml_rf(
                    n_estimators=trial.suggest_int("n_estimators", 100, 1000),
                    max_depth=trial.suggest_int("max_depth", 4, 10)
                )
            else:
                model = MultiOutputRegressor(
                    cuml_rf(
                        n_estimators=trial.suggest_int("n_estimators", 100, 1000),
                        max_depth=trial.suggest_int("max_depth", 4, 10)
                    )
                )
        '''
        Treina o modelo e calcula o RMSE das previsões.
        '''
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        rmse = mean_squared_error(y_test, preds, squared=False)
        print(f"Model {model_name} RMSE: {rmse}")
        return model, rmse
    except Exception as e:
        print(f"Error in train_and_evaluate for model {model_name}: {str(e)}")
        raise

'''
Função `cross_validate_and_save` realiza a validação cruzada e salva o melhor modelo treinado. 
Recebe o caminho do arquivo de dados, as colunas alvo e o tipo de previsão como parâmetros.
'''
def cross_validate_and_save(file_path, target_cols, prediction_type):
    try:
        print(f"Starting cross-validation for file: {file_path}")
        df = cudf.read_csv(file_path)
        
        '''
        Verifica se o DataFrame foi carregado corretamente e possui as colunas esperadas.
        '''
        if df.empty:
            print(f"Dataframe loaded from {file_path} is empty.")
            return
        if not all(col in df.columns for col in ['hour_from_file_train', 'date_from_file']):
            print(f"Missing expected columns in dataframe loaded from {file_path}. Columns found: {df.columns}")
            return

        print(f"Columns in dataframe: {df.columns}")
        
        file_info = {
            'hour_from_file_train': df['hour_from_file_train'].unique().to_arrow().to_pylist(),
            'date_from_file': df['date_from_file'].unique().to_arrow().to_pylist()
        }
        
        '''
        Define as colunas a serem descartadas com base no tipo de previsão.
        '''
        if prediction_type == 'lat_long':
            drop_cols = ['datahora_converted', 'velocidade', 'latitude_diff', 'longitude_diff', 'distancia', 'day_of_week', 'hour', 'minute']
        else:
            drop_cols = ['velocidade', 'datahora_converted', 'day_of_week', 'hour', 'minute', 'diff_timestamp', 'latitude_diff', 'longitude_diff', 'distancia']
        
        print(f"Dropping columns: {drop_cols}")
        df = df.drop(columns=drop_cols)
        
        hour_from_file_train_1, hour_from_file_train_2 = file_info['hour_from_file_train']
        date_from_file = file_info['date_from_file'][0]

        final_model_file = os.path.join(final_model_path, f"final_model_predict_{prediction_type}_{date_from_file}_{hour_from_file_train_1}_{hour_from_file_train_2}.pkl")
        
        '''
        Verifica se o modelo final já existe para evitar a reprocessamento.
        '''
        if os.path.exists(final_model_file):
            print(f"Final model for {file_path} already exists. Skipping cross-validation.")
            return

        train_data = df[df['hour_from_file_train'] == hour_from_file_train_1]
        test_data = df[df['hour_from_file_train'] == hour_from_file_train_2]

        if train_data.empty or test_data.empty:
            print(f"Train or test data is empty for {file_path}.")
            return

        print(f"Train data shape: {train_data.shape}, Test data shape: {test_data.shape}")

        '''
        Separa os dados de treino e teste em features (`X_train`, `X_test`) e targets (`y_train`, `y_test`).
        '''
        X_train = train_data.drop(columns=target_cols + ['hour_from_file_train', 'date_from_file', 'date_hour_from_file_train'])
        y_train = train_data[target_cols]
        X_test = test_data.drop(columns=target_cols + ['hour_from_file_train', 'date_from_file', 'date_hour_from_file_train'])
        y_test = test_data[target_cols]

        feature_names = X_train.columns.to_list()

        X_train_np = X_train.to_numpy()
        y_train_np = y_train.to_numpy()
        X_test_np = X_test.to_numpy()
        y_test_np = y_test.to_numpy()

        best_model = None
        best_rmse = float('inf')
        best_params = None
        best_model_name = None

        '''
        Loop que percorre os modelos a serem avaliados (CatBoost, XGBoost, cuml_rf).
        '''
        for model_name in tqdm(["CatBoost", "XGBoost", "cuml_rf"], desc="Models", leave=False):
            '''
            Define a função objetivo para a otimização com o Optuna.
            '''
            def objective(trial):
                model, rmse = train_and_evaluate(X_train_np, X_test_np, y_train_np, y_test_np, model_name, trial, len(target_cols))
                return rmse

            study = optuna.create_study(direction="minimize")
            study.optimize(objective, n_trials=10)
            best_trial = study.best_trial

            model, rmse = train_and_evaluate(X_train_np, X_test_np, y_train_np, y_test_np, model_name, best_trial, len(target_cols))

            '''
            Atualiza o melhor modelo se o RMSE atual for menor que o melhor RMSE encontrado até então.
            '''
            if rmse < best_rmse:
                best_rmse = rmse
                best_model = model
                best_params = best_trial.params
                best_model_name = model_name

            intermediate_model_file = os.path.join(intermediate_model_path, f"{model_name}_{prediction_type}_{date_from_file}_{hour_from_file_train_1}_{hour_from_file_train_2}.pkl")
            joblib.dump(model, intermediate_model_file)
            print(f"Saved intermediate model for {model_name} at {intermediate_model_file}")

        '''
        Re-treina o melhor modelo com os dados de treino completos.
        '''
        print(f"Re-training best model {best_model_name} on entire training data")

        if best_model_name == "CatBoost":
            final_model = MultiOutputRegressor(
                CatBoostRegressor(
                    iterations=best_params["iterations"],
                    learning_rate=best_params["learning_rate"],
                    depth=best_params["depth"],
                    silent=True,
                    task_type="GPU"
                )
            )
        elif best_model_name == "XGBoost":
            final_model = MultiOutputRegressor(
                XGBRegressor(
                    n_estimators=best_params["n_estimators"],
                    learning_rate=best_params["learning_rate"],
                    max_depth=best_params["max_depth"],
                    tree_method='hist',
                    device='cuda'
                )
            )
        elif best_model_name == "cuml_rf":
            if len(target_cols) == 1:
                final_model = cuml_rf(
                    n_estimators=best_params["n_estimators"],
                    max_depth=best_params["max_depth"]
                )
            else:
                final_model = MultiOutputRegressor(
                    cuml_rf(
                        n_estimators=best_params["n_estimators"],
                        max_depth=best_params["max_depth"]
                    )
                )

        '''
        Treina o modelo final com os dados de treino completos.
        '''
        final_model.fit(X_train_np, y_train_np)

        '''
        Salva o modelo final juntamente com os nomes das features.
        '''
        joblib.dump({'model': final_model, 'feature_names': feature_names}, final_model_file)
        print(f"Best model for {file_path} is {best_model_name} with RMSE: {best_rmse}")

    except Exception as e:
        print(f"Error in cross_validate_and_save for file {file_path}: {str(e)}")
        raise

'''
Processa os arquivos de acordo com o tipo de previsão (`lat_long` ou `datahora`).
'''
def process_files(prediction_type):
    '''
    Define o sufixo do arquivo e as colunas alvo com base no tipo de previsão.
    '''
    if prediction_type == 'lat_long':
        file_ending = 'datahora.csv'
        target_cols = ['latitude', 'longitude']
    else:
        file_ending = 'lat_long.csv'
        target_cols = ['datahora']
    
    files = [f for f in os.listdir(base_path) if f.startswith("preprocessed_train_data") and f.endswith(file_ending)]
    '''
    Percorre cada arquivo e chama a função `cross_validate_and_save` para processar e validar o arquivo.
    '''
    for file_name in tqdm(files, desc=f"Processing files for {prediction_type}"):
        file_path = os.path.join(base_path, file_name)
        try:
            print(f"Processing file: {file_name}")
            cross_validate_and_save(file_path, target_cols, prediction_type)
        except Exception as e:
            print(f"Error processing file {file_name}: {str(e)}")

if __name__ == "__main__":
    process_files('lat_long')
    process_files('datahora')


<h1>Realizando as previsões</h1>
<p>
<li>Todos os dados usados foram coletados exclusivamente do último post em relação à tarefa 3 no Moodle</li>
<li>Os dados foram treinados exclusivamente usando os dados criados a partir dos arquivos .JSON cujo nome começa com 2024</li>
<li>Para fazer a avaliação do modelo durante a validação cruzada, os datasets de treino foram divididos entre treino e test, como normalmente é feito</li>
<li>Os modelos criados durante a fase de treinamento foram utilizados nos datasets de teste disponibilizados no último link.</li>
<li>Como previamente os datasets de treino haviam sido divididos para aplicar os modelos de previsão de datahora e para aplicar os modelos de previsão de latitude e longitude, colunas específicas foram adicionadas ou retiradas aos dois datasets de teste</li>
<li>Para o dataset de teste onde o modelo de prever a datahora foi aplicado, as colunas temporais foram retiradas para evitar o data leakage</li>
<li>Para o dataset de teste onde o modelo de prever latitude e longitude foi aplicado, as colunas com informações geoespaciais foram retiradas para evitar o data leakage.</li>
<li>Após a previsão foram criados dois arquivos: um contendo todas as colunas anteriores, mais as previsões de datahora chamado predicted_datahora e outro contendo todas as colunas anteriores mais as previsões de latitude e longitude chamado predicted_lat_long</li>
 </p>

In [ ]:
'''
Logging
'''
logging.basicConfig(level=logging.INFO, filename='prediction.log', filemode='w',
                    format='%(name)s - %(levelname)s - %(message)s')

'''
Define os caminhos para os diretórios base, modelo final e respostas.
'''
base_path = 'dados_finais/feature_modeling'
final_model_path = 'dados_finais/cross_validation/final_model'
respostas_path = 'dados_finais/respostas'

'''
Garante que os diretórios para salvar as respostas existem, criando-os se necessário.
'''
os.makedirs(respostas_path, exist_ok=True)

'''
Realiza a previsão dos valores utilizando um modelo treinado. 
Recebe como parâmetros o arquivo de teste, o arquivo do modelo, as colunas alvo e o arquivo de saída 
para salvar as previsões.
'''
def predict_values(test_file, model_file, target_cols, output_file):
    logging.info(f"Processing file {test_file} with model {model_file}")
    
    '''
    Lê o arquivo CSV de teste.
    '''
    df = cudf.read_csv(test_file)
    logging.info(f"Initial shape of the dataframe: {df.shape}")

    '''
    Salva e remove as colunas necessárias.
    '''
    date_hour_from_file_test = df['date_hour_from_file_test'].unique()[0]
    id_column = df['id']
    
    '''
    Lista de colunas a serem removidas.
    '''
    drop_columns = ['hour_from_file_test', 'date_hour_from_file_test', 'date_from_file', 'datahora_converted', 'id']
    
    '''
    Verifica se as colunas existem antes de removê-las.
    '''
    existing_columns = [col for col in drop_columns if col in df.columns]
    df = df.drop(columns=existing_columns)
    
    logging.info(f"Shape of the dataframe after dropping columns: {df.shape}")

    '''
    Carrega o modelo e os nomes das features.
    '''
    model_data = joblib.load(model_file)
    model = model_data['model']
    feature_names = model_data['feature_names']

    '''
    Verifica se há features faltantes no conjunto de teste.
    '''
    missing_features = set(feature_names) - set(df.columns)
    if missing_features:
        logging.error(f"Missing features in test data: {missing_features}")
        raise ValueError(f"Missing features in test data: {missing_features}")

    '''
    Reordena as colunas para corresponder aos dados de treino.
    '''
    df = df[feature_names]
    
    '''
    Converte para pandas para compatibilidade com a previsão do modelo.
    '''
    df_pd = df.to_pandas()

    '''
    Realiza as previsões.
    '''
    predictions = model.predict(df_pd)
    logging.info(f"Predictions made. Shape of predictions: {predictions.shape}")

    '''
    Garante que as previsões tenham a forma correta.
    '''
    if isinstance(predictions, pd.Series):
        predictions = predictions.to_numpy()

    if predictions.ndim == 1:
        predictions = predictions.reshape(-1, 1)
    
    '''
    Adiciona as previsões ao DataFrame.
    '''
    for i, col in enumerate(target_cols):
        if i < predictions.shape[1]:
            df[col] = predictions[:, i]
        else:
            logging.error(f"Predictions do not have enough columns for target {col}")
            raise ValueError(f"Predictions do not have enough columns for target {col}")

    '''
    Insere novamente a coluna `id` no DataFrame.
    '''
    df.insert(0, 'id', id_column)
    logging.info(f"Shape of the dataframe after adding predictions: {df.shape}")

    '''
    Salva o arquivo final de previsões.
    '''
    df.to_csv(output_file, index=False)
    logging.info(f"Predicted values saved to {output_file}")

'''
Processa os arquivos de teste para previsões de `datahora`. 
Ela procura por modelos correspondentes e realiza previsões, salvando os resultados.
Modelos correspondentes são aqueles cujo nome possuem a mesma data extraída do arquivo zip e a mesma
hora extraída do arquivo json.
'''
def process_datahora_predictions():
    test_files = [f for f in os.listdir(base_path) if f.startswith("preprocessed_treated_test_lat_long") and f.endswith(".csv")]
    model_files = [f for f in os.listdir(final_model_path) if f.startswith("final_model_predict_datahora") and f.endswith(".pkl")]

    '''
    Percorre cada arquivo de teste e realiza previsões usando o modelo correspondente.
    '''
    for test_file in tqdm(test_files, desc="Processing datahora predictions"):
        date, hour_from_file_test = test_file.split('_')[-2:]
        hour_from_file_test = hour_from_file_test.split('.')[0]
        date_hour_from_file_test = f"{date}_{hour_from_file_test}"
        output_file = os.path.join(respostas_path, f"resposta_predicted_datahora_{date_hour_from_file_test}.csv")

        '''
        Verifica se o arquivo de previsão já existe para evitar processamento repetido.
        '''
        if os.path.exists(output_file):
            logging.info(f"Prediction file {output_file} already exists. Skipping.")
            continue

        matching_model = None
        '''
        Procura pelo modelo correspondente ao arquivo de teste.
        '''
        for model_file in model_files:
            model_date, hour1, hour2 = model_file.split('_')[-3:]
            hour2 = hour2.split('.')[0]
            if model_date == date and int(hour_from_file_test) == int(hour2) + 1:
                matching_model = os.path.join(final_model_path, model_file)
                break

        '''
        Se encontrar um modelo correspondente, realiza a previsão; caso contrário, registra um erro.
        '''
        if matching_model:
            try:
                predict_values(os.path.join(base_path, test_file), matching_model, ['datahora'], output_file)
            except Exception as e:
                logging.error(f"Error processing file {test_file} with model {matching_model}: {e}")
                raise
        else:
            logging.error(f"No matching model found for test file {test_file}")
            raise ValueError(f"No matching model found for test file {test_file}")

'''
Processa os arquivos de teste para previsões de `latitude` e `longitude`. 
Ela procura por modelos correspondentes e realiza previsões, salvando os resultados.
'''
def process_lat_long_predictions():
    test_files = [f for f in os.listdir(base_path) if f.startswith("preprocessed_treated_test_datahora") and f.endswith(".csv")]
    model_files = [f for f in os.listdir(final_model_path) if f.startswith("final_model_predict_lat_long") and f.endswith(".pkl")]

    '''
    Loop que percorre cada arquivo de teste e realiza previsões usando o modelo correspondente.
    '''
    for test_file in tqdm(test_files, desc="Processing lat_long predictions"):
        date, hour_from_file_test = test_file.split('_')[-2:]
        hour_from_file_test = hour_from_file_test.split('.')[0]
        date_hour_from_file_test = f"{date}_{hour_from_file_test}"
        output_file = os.path.join(respostas_path, f"resposta_predicted_lat_long_{date_hour_from_file_test}.csv")

        '''
        Verifica se o arquivo de previsão já existe para evitar processamento repetido.
        '''
        if os.path.exists(output_file):
            logging.info(f"Prediction file {output_file} already exists. Skipping.")
            continue

        matching_model = None
        '''
        Procura pelo modelo correspondente ao arquivo de teste.
        '''
        for model_file in model_files:
            model_date, hour1, hour2 = model_file.split('_')[-3:]
            hour2 = hour2.split('.')[0]
            if model_date == date and int(hour_from_file_test) == int(hour2) + 1:
                matching_model = os.path.join(final_model_path, model_file)
                break

        '''
        Se encontrar um modelo correspondente, realiza a previsão; caso contrário, registra um erro.
        '''
        if matching_model:
            try:
                predict_values(os.path.join(base_path, test_file), matching_model, ['latitude', 'longitude'], output_file)
            except Exception as e:
                logging.error(f"Error processing file {test_file} with model {matching_model}: {e}")
                raise
        else:
            logging.error(f"No matching model found for test file {test_file}")
            raise ValueError(f"No matching model found for test file {test_file}")


if __name__ == "__main__":
    process_lat_long_predictions()
    process_datahora_predictions()


<h1>Criando os arquivos de submissão</h1>
<p>Como especificado no enunciado, o formato dos arquivos de submissão deveriam ser em .json. Então eu fiz um arquivo .json para cada previsão feita. <b>O grande problema com esse approach é que não será possível submeter essas respostas, haja vista que isso gerou arquivos .json muito grande. </b> </p>




In [ ]:
#Para a previsão de datahora
'''
{ 
   "aluno": "Your Name",
   "datahora": "2024-05-20 13:00:00" #Mesmo valor do nome do arquivo .json,
   "previsoes": [ 
                  [400172783234, 1716205511000] #Id e a previsão de datahora previstos, 
                  [282474448123, 1716204264000]
                ], 
   "senha": "your_password"
}
'''

In [ ]:
#Para a previsão de latitude e longitude
'''
{ 
   "aluno": "Your Name",
   "datahora": "2024-05-20 13:00:00" #Mesmo valor do nome do arquivo .json,
   "previsoes": [ 
                  [362511850614, -22.82553, -43.16925] #Id e valores de latitude e longitude previstos, 
                  [288961216441, -23.0202, -43.46159]
                ], 
   "senha": "your_password"
}
'''

In [ ]:
'''
Define os caminhos para os diretórios base e de submissão, garantindo que os diretórios existam.
'''
root_path = os.getcwd()
base_path = os.path.join(root_path, 'dados_finais', 'respostas')
submission_path = os.path.join(base_path, 'submission')

os.makedirs(submission_path, exist_ok=True)

'''
Função `create_json_from_csv` lê um arquivo CSV, extrai as informações necessárias e cria um arquivo JSON com o formato especificado.
Recebe como parâmetros o arquivo CSV, o nome do aluno e a senha.
'''
def create_json_from_csv(csv_file, aluno, senha):
    df = pd.read_csv(csv_file)

    '''
    Extrai a data e hora do nome do arquivo.
    '''
    file_name = os.path.basename(csv_file)
    date_hour_part = file_name.split('_')[-2] + '_' + file_name.split('_')[-1].split('.')[0]
    datahora_datetime = pd.to_datetime(date_hour_part, format='%Y-%m-%d_%H').strftime('%Y-%m-%d %H:00:00')

    '''
    Determina o tipo de previsão com base no nome do arquivo.
    '''
    if 'resposta_predicted_datahora_' in csv_file:
        prediction_type = 'datahora'
        json_file = os.path.join(submission_path, os.path.basename(csv_file).replace('resposta_predicted_', '').replace('.csv', '.json'))
        previsoes = df[['id', 'datahora']].values.tolist()

    elif 'resposta_predicted_lat_long_' in csv_file:
        prediction_type = 'lat_long'
        json_file = os.path.join(submission_path, os.path.basename(csv_file).replace('resposta_predicted_', '').replace('.csv', '.json'))
        previsoes = df[['id', 'latitude', 'longitude']].values.tolist()

    else:
        raise ValueError(f"Unknown prediction type for file {csv_file}")

    '''
    Cria a estrutura do JSON.
    '''
    # Create the JSON structure
    json_data = {
        "aluno": aluno,
        "datahora": datahora_datetime,
        "previsoes": previsoes,
        "senha": senha
    }

    '''
    Salva o arquivo JSON.
    '''
    with open(json_file, 'w') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)

    print(f"Created {json_file}")

'''
Percorre todos os arquivos CSV na pasta base e chama a função `create_json_from_csv` para cada um.
Recebe como parâmetros o nome do aluno e a senha.
'''
def process_csv_files(aluno, senha):
    csv_files = [f for f in os.listdir(base_path) if f.startswith("resposta_predicted_") and f.endswith(".csv")]
    
    '''
    Percorre cada arquivo CSV e chama a função `create_json_from_csv`.
    '''
    for csv_file in tqdm(csv_files, desc="Processing CSV files"):
        csv_file_path = os.path.join(base_path, csv_file)
        create_json_from_csv(csv_file_path, aluno, senha)
''' 
Executa tudo
'''
if __name__ == "__main__":
    aluno = "Wagner Luiz Lobo Ferreira"
    senha = "muramassa_coi"
    
    process_csv_files(aluno, senha)


<h1>Sending the answers via API</h1>
<p>Para enviar os arquivos .json contendo as previsões, eu usei uma API disponibilizada pelo professor, dentro de um loop.</p>

In [ ]:
'''
Define o diretório contendo os arquivos JSON que serão enviados.
'''
base_dir = os.getcwd()
resposta_dir = os.path.join(base_dir, 'dados_finais', 'respostas', 'submission')

'''
Define o endpoint da API para onde os arquivos JSON serão enviados.
'''
api_url = 'https://barra.cos.ufrj.br:443/rest/rpc/avalia'

'''
Define as credenciais do aluno que serão usadas na autenticação dos envios.
'''
aluno = "Wagner Luiz Lobo Ferreira"
senha = "muramassa_coi"

'''
Função `send_json_file` envia um arquivo JSON para a API. 
Recebe como parâmetro o caminho do arquivo JSON.
'''
def send_json_file(file_path):
    '''
    Abre e carrega o conteúdo do arquivo JSON.
    '''
    with open(file_path, 'r') as json_file:
        json_data = json.load(json_file)
    
    '''
    Adiciona as credenciais do aluno aos dados JSON.
    '''
    json_data['aluno'] = aluno
    json_data['senha'] = senha
    
    '''
    Loop que tenta enviar o arquivo JSON para a API até obter uma resposta de sucesso ou falha definitiva.
    '''
    while True:
        try:
            '''
            Envia uma requisição POST para a API com os dados JSON.
            '''
            response = requests.post(
                api_url,
                headers={
                    'accept': 'application/json',
                    'Content-Type': 'application/json'
                },
                data=json.dumps(json_data),
                timeout=10 
            )
            '''
            Verifica se a resposta da API foi bem-sucedida (código 200).
            '''
            if response.status_code == 200:
                print(f"File {os.path.basename(file_path)} sent successfully.")
                break
            else:
                print(f"Failed to send file {os.path.basename(file_path)}. Status code: {response.status_code}, Response: {response.text}")
                break
        except requests.exceptions.RequestException as e:
            '''
            Em caso de erro, aguarda 5 segundos antes de tentar novamente.
            '''
            print(f"Error sending file {os.path.basename(file_path)}: {e}. Retrying in 5 seconds...")
            time.sleep(5)

'''
Itera sobre todos os arquivos JSON no diretório e envia cada um deles.
'''
for file_name in os.listdir(resposta_dir):
    if file_name.endswith('.json'):
        file_path = os.path.join(resposta_dir, file_name)
        send_json_file(file_path)
